<center><h1><FONT FACE="Vollkorn"><Font Color=navy> Analysis of Suppliers Catalogs</h1> 
<center><h2> <OL Type = ' '>
Gaining visibility of supplier pricing and determining potential impact on the 
<Center>University of California, Berkeley and San Francisco
<p>
<center><img src="http://calwatchdog.com/wp-content/uploads/2013/09/University_of_California_seal.png" >

<Font FACE= Times><Font Size= 4> <b>Introduction<p> </b>
<p>
       The University of California, Berkeley and San Francisco have strategic relationships with core suppliers through which its students, faculty, and staff do routine business. The daily operations of UCSF and UC Berkeley are fueled by the constant procurement of goods and services off of an e-commerce system called BearBuy. It is through this system that suppliers are enabled to host product catalogs. The Procurement department is responsible for determining the impact of the suggested pricing. Through an analysis of the proposed catalogs, the University gains visibility into any future price increases or product removals and determines if the proposed catalog is acceptable to be purchased off of.
           <p>
<t>
<b> The Analysis:</b>
<p>  <t>    The several hosted catalogs all offer an abundant amount of products, ranging from a few thousand to over a million line items. In order to understand relevant product changes, the previous 12 months worth of campus spend through that supplier is matched up against both the current and proposed catalog. This narrows down the entirety of the catalog to the relevant products of which the campus purchases. From the past 12 month spend, the analysis determines whether or not there are price increases through the proposed catalog and if the supplier is following the terms of their contract. 
    
<p>Currently, the analysis does not take into account products of which the university has never purchased. The simple assumption that past historical spend accurately reflects forecasted purchases allows for an opportunity in an improvement in the analysis.</p><br>
        
<b>Benefits of Using Python/PANDAS:</b>
<p>
The routine review of proposed catalogs, the limits of Excel and time consumption of repeated manual analysis make an automated process that can  analyze and tackle the large amount data extremeley valuable. Python, and more specifically, PANDAS, is an excellent tool to run this continued analysis of determining potential price increases from the suppliers.
<p>
    PANDAS allows for the easy manipulation of dataframes and the content within. The analyst inputs the correct file location of the catalogs to be analyzed as well as notes the specific campus and supplier. The code uses this data to ouptput all of the necessary content for determining if a price file is appropriate into a running log. This not only yields instant visibility into the impact of a catalog, but continuously organizes the results chronologically for easy recall.
    
<p><i>Alexis Perez, 2014<br>
Andrew Clark, 2015
</i></p>

In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame as DF
import csv

#Working directory
wd = 'C:/Users/aclark/Desktop/CatTest/'

#File Names
CurrentCatalogName = 'PROMEGA_V15.txt'
ProposedCatalogName = 'PROMEGA_V16.txt'
SpendFileName = 'POData_Promega.csv'

#Campus and Supplier Name
Campus= 'UCSF'
Supplier = 'Promega'
 
#Catalog Analysis Log File
LogFile = 'C:/Users/aclark/Desktop/CatTest/CatalogAnalysisLog.csv'

In [2]:
#Load the files into Memory
Current = pd.read_table(wd + CurrentCatalogName, error_bad_lines = False, low_memory=False, thousands=',')
Proposed = pd.read_table(wd + ProposedCatalogName, error_bad_lines = False,  low_memory=False, thousands=',')
Spend= pd.read_csv(wd + SpendFileName, error_bad_lines = False,  low_memory=False, thousands=',')

#Modify the Spend Columns to match the Catalog Columns for ease of merging and data management
Spend = Spend.rename(columns={'Amount/UOM & UOM':'Packaging UOM','SKU/Catalog #':'Part Number'})

#In Amount/UOM & UOM the unit of measure might be listed as 1/EA. This won't match up if in the Price catalog it is listed as EA
#Change each UOM column to only the LAST two characters to ensure similarities in format:
# Create a new Column called PartUOM so we can compare appropriately between all three files
def ac_clean_up(df):
    #Rename the words into abbrivations for consistency
    df['Packaging UOM'] = df['Packaging UOM'].replace({'Packaging UOM':{'.*box.*':'BX',
                                                                        '.*package.*':'PK',
                                                                        '.*case.*':'CS',
                                                                        '.*each.*':'EA'
                                                                        'CS':'CA'}})

    #Assign this column as just the last two chars of the column
    df['Packaging UOM'] = df['Packaging UOM'].astype('str').str[-2:] 
    
    #Create the PartUOM column using the part number and cleaned packaging UOM
    df['PartUOM']=df.apply(lambda x:'%s,%s' % (x['Part Number'],x['Packaging UOM']),axis=1)  
    
    #Further clean the PartUOM to remove spaces, hyphens, commas, and other unnecessary columns
    df['PartUOM']= df['PartUOM'].str.replace("-", "") \
                                .str.replace(",","") \
                                .str.replace("/","") \
                                .str.replace("_","") \
                                .str.replace(".","") \
                                .str.replace("'","")

#Create a list of files and then loop over the list to apply the clean-up function over all items in the list.  
List=(Spend, Current, Proposed)

for x in List:
    ac_clean_up(x)
    
#Group Spend data by PartUOM and sum the quantities, spend, etc.
SpendPivot = Spend.groupby(['PartUOM']).agg({'Quantity':np.sum,
                                             'Extended Price': np.sum})

# Merge the Proposed Catalog with the Current Catalog
# Create a new dataframe with the specific columns necessary for the analysis
comparison = pd.merge(Current, Proposed, on='PartUOM', how='inner', suffixes=('_Current', '_Proposed')) 
comparison = comparison.loc[:,['PartUOM', 'Price_Current' , 'Price_Proposed']].set_index('PartUOM')

#In order to properly analyze the data, we only want SKUs where they are not blank. Otherwise there is no way to tell if they should have been in a catalog
#This line of code omits all SKUS that are blank
Spend = Spend[Spend['Part Number'] >0]
Spend = Spend[Spend['Part Number'].notnull()]

#Want to get rid of SKUs where the person entered N/A. However, we don't use .str.contains because N/A COULD be a part of the SKU/Part #
List = ['N.A.','N/A','NA','n/a','n.a.','na']

#Using .isin will filter out those SKUs where it exactly equals N/A isntead of just contains it.
Spend = Spend[-Spend['Part Number'].isin(List)]

#Using str.contains allows us to grab anything which references these words
Spend = Spend[-Spend['Part Number'].str.contains('Quote | see attach', case=False)]

#Merge the two catalog comparisons with the historic spend based on PartUOM
#Merging on 'Left' means we want all values of the Spend and only the values of the comparison that match.
analysis = pd.merge(SpendPivot, comparison, left_index=True, right_index=True, how='left') 

#Extract only the columns that are relevant to the analysis
analysis = analysis.loc[:,['UNSPSC','Manufacturer', 'Quantity', 'Unit Price', 'Extended Price', 'Price_Current' , 'Price_Proposed','Item Type']]

###########If you only want items that are SQ Hosted Product##########################################################################
##Hashtag this first
#analysis= analysis[analysis['Item Type'] != 'NonCatalog Product']

##If there are any prices removed in the catalogs, the vendors will either put 'Price Removed' in the Proposed catalog under Current or Proposed Prices

#For the rest of the analysis we don't want 'Price Removed' since this is a string and we can't do operations on the entire column
# We do want to know how many items were removed
##  We can compute that now, and then later change the 'Price Removed' to 0 so we can do calculations
#Here we sum the number of times Price Removed shows up in either of two columns to get the number of deleted items

analysis['Price_Proposed'] = analysis['Price_Proposed'].astype('str')
analysis['Price_Current'] = analysis['Price_Current'].astype('str')
Removed_Proposed = analysis['Price_Proposed'].str.contains('Price Removed').sum()
Removed_Current = analysis['Price_Current'].str.contains('Price Removed').sum()

#I put two variables because the vendor could have the Price removed in Current or Proposed Price
print "The number of deleted items in proposed is %s and in current is %s" %(Removed_Proposed,Removed_Current)

The number of deleted items in proposed is 0 and in current is 0


In [3]:
#Set the analysis file to the correct datatypes
analysis[['Price_Current','Price_Proposed']] = analysis[['Price_Current','Price_Proposed']].astype(float)

#Append computed values at the end of the DataFrame that helps us in the final analysis 
#Compute Ext. Current Price, Proposed Ext. Price, $ Difference and % Difference
analysis['Current Ext. Price'] = analysis['Quantity'] * analysis['Price_Current']
analysis['Proposed Ext. Price'] = analysis['Quantity'] * analysis['Price_Proposed']

analysis['$ Difference'] = analysis['Proposed Ext. Price']-analysis['Current Ext. Price']
analysis['% Difference'] = analysis['$ Difference']/analysis['Current Ext. Price']

# Let's add a validity column
# Validity answers the question, how much the comparable spend purchased is reflected in the current and proposed catalogs
# Recall, we already took out part numbers that were null or mislabeled by the user. 
# We don't want this data since it won't accurately reflect an actual product.
# A SKU that is blank or mislabeled shouldn't be included in the analysis
analysis['Valid'] = analysis['Price_Current'].notnull() * analysis['Price_Proposed'].notnull()

## Now we can multiply the Extended prices by the Validity column to get the valid spend, valid current and proposed ext. prices
analysis['Valid Spend'] = analysis['Extended Price'] * analysis['Valid']
analysis['Valid Current Ext. Price'] = analysis['Current Ext. Price'] * analysis['Valid']
analysis['Valid Proposed Ext. Price'] = analysis['Proposed Ext. Price'] * analysis['Valid']

#This locale allows us to convert floats into currency
import locale
locale.setlocale( locale.LC_ALL, '' )

##Sum up the column values to get a total price 
Spend = analysis['Extended Price'].sum()
Valid_Spend = analysis['Valid Spend'].sum()

# We can get the Validity percentage
Validity_percentage = Valid_Spend / Spend
valid_percentage = '{percent:.2%}'.format(percent= Validity_percentage)

#Sum up Current and Proposed Extended Prices
Valid_CurrExt_Price = analysis['Valid Current Ext. Price'].sum()
Valid_ProposedExt_Price = analysis['Valid Proposed Ext. Price'].sum()

Total_Catalog_Price_Difference = Valid_ProposedExt_Price - Valid_CurrExt_Price
Percent_Increase = Total_Catalog_Price_Difference/Valid_CurrExt_Price

#Understand the count of different SKU's purchased
Count_SKUS = len(analysis.index)

#set these variables equal to currency so the end result is easier to read
spenddollars = locale.currency(Spend)
valid_spend_dollars = locale.currency(Valid_Spend)
current_price_dollars = locale.currency (Valid_CurrExt_Price)
proposed_price_dollars = locale.currency(Valid_ProposedExt_Price)
price_difference_dollars = locale.currency(Total_Catalog_Price_Difference)

#Now we want the percentage to actually look like a percentage
#Ex. .0007 will simply yield 0 if we don't format it properly
percentage = '{percent:.2%}'.format(percent=Percent_Increase)

#Notice how the variables used are %s not %d. The locale.currency function changes the float into a string.
print "The Total spend is %s and the valid spend is %s" %(spenddollars,valid_spend_dollars)
print "Validity percentage is %s \n" %valid_percentage
print "The number of SKUS analyzed is %d\n" %Count_SKUS
print "The Total Valid Current Extended price is %s" %current_price_dollars
print "The Total Valid Proposed Extended price is %s\n\n" %proposed_price_dollars
print "The dollar difference between Total Proposed and Total Current is %s" %price_difference_dollars
print "The percent increase is %s" %percentage

The Total spend is $529977.28 and the valid spend is $472727.66
Validity percentage is 89.20% 

The number of SKUS analyzed is 297

The Total Valid Current Extended price is $479216.45
The Total Valid Proposed Extended price is $479216.45


The dollar difference between Total Proposed and Total Current is $0.00
The percent increase is 0.00%


C:\Anaconda\lib\site-packages\pandas\computation\expressions.py:190: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  unsupported[op_str]))


In [4]:
analysis

,Part Number,PartUOM,UNSPSC,Manufacturer,Quantity,Unit Price,Extended Price,Price_Current,Price_Proposed,Item Type,Current Ext. Price,Proposed Ext. Price,$ Difference,% Difference,Valid,Valid Spend,Valid Current Ext. Price,Valid Proposed Ext. Price
PartUOM,,,,,,,,,,,,,,,,,,
A1071EA,NaN,NaN,NaN,NaN,1,NaN,100.10,100.10,100.10,NaN,100.10,100.10,0,0,True,100.10,100.10,100.10
A1120EA,NaN,NaN,NaN,NaN,1,NaN,116.20,121.10,121.10,NaN,121.10,121.10,0,0,True,116.20,121.10,121.10
A1125EA,NaN,NaN,NaN,NaN,1,NaN,298.20,298.20,298.20,NaN,298.20,298.20,0,0,True,298.20,298.20,298.20
A1222EA,NaN,NaN,NaN,NaN,9,NaN,2201.50,245.70,245.70,NaN,2211.30,2211.30,0,0,True,2201.50,2211.30,2211.30
A1223EA,NaN,NaN,NaN,NaN,7,NaN,796.60,114.10,114.10,NaN,798.70,798.70,0,0,True,796.60,798.70,798.70
A1360EA,NaN,NaN,NaN,NaN,13,NaN,1712.00,132.80,132.80,NaN,1726.40,1726.40,0,0,True,1712.00,1726.40,1726.40
A1441EA,NaN,NaN,NaN,NaN,1,NaN,28.70,29.40,29.40,NaN,29.40,29.40,0,0,True,28.70,29.40,29.40
A1460EA,NaN,NaN,NaN,NaN,9,NaN,1948.80,218.40,218.40,NaN,1965.60,1965.60,0,0,True,1948.80,1965.60,1965.60
A159CCEA,NaN,NaN,NaN,NaN,1,NaN,18.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0.00,NaN,NaN


In [26]:
#Let's now get the spend for any products that were removed
#We can create a separate dataframe where the ProposedPrice is 0 AND there exists a Current Price

#First we need to fill the Proposed Prices to 0 if it is empty
analysis['Price_Proposed'] = analysis['Price_Proposed'].fillna(0)

#After filling in the empty spaces with 0 we create a dataframe where the Proposed Price is ONLY zero
proposed_0=  analysis[analysis['Price_Proposed'] == 0]

#We don't want both Current and Proposed to be zero
#If there exists a current price and not a proposed then it shows that a price was removed
#Create a dataframe where Proposed Price is 0 and Current price is some number 
currentnot_zero = proposed_0[proposed_0['Price_Current'] >0]

#This variable will show the aggregated historic spend on the products that were removed
Removed_Spend = currentnot_zero['Extended Price'].sum()

#Let's put this value in currency notation
Removed_Spend_Dollars = locale.currency(Removed_Spend)

print "The Removed Spend is %s" %Removed_Spend_Dollars

The Removed Spend is $0.00


In [28]:
#Create a pivot table to look at top UNSPSC's and specific numbers associated with those product categories

from pandas import pivot_table
import numpy as np
  
UNSPSC = pivot_table(analysis, values = ['Extended Price', 'Quantity', 
                                         '$ Difference','Valid Current Ext. Price',
                                         'Valid Proposed Ext. Price'], index = ['UNSPSC'], aggfunc = np.sum)

UNSPSC['Percent Increase'] = UNSPSC['$ Difference'] / UNSPSC['Valid Current Ext. Price']
UNSPSC['Percent Increase']= UNSPSC['Percent Increase']*100

#Want to show the pivot table with the top UNSPSC's by spend
#Use .head() because some pivot tables will be too large to show all of the columns and we only care about the top 5 anyway
UNSPSC=UNSPSC.rename(columns={'Extended Price':'Spend'})
UNSPSC.sort("Spend", ascending=False).head()

,$ Difference,Spend,Quantity,Valid Current Ext. Price,Valid Proposed Ext. Price,Percent Increase
UNSPSC,,,,,,
41116111,NaN,89522.68,199,NaN,NaN,NaN
41106513,NaN,78574.94,160,NaN,NaN,NaN
41106502,NaN,43503.02,108,NaN,NaN,NaN
41105500,NaN,34724.00,153,NaN,NaN,NaN
12352204,NaN,30402.38,274,NaN,NaN,NaN


In [29]:
#Create a pivot table to look at the summary of the proposed catalog impact by Manufacturer

Manufacturer = pivot_table(analysis, values = ['Extended Price', 'Quantity', 
                                               '$ Difference','Valid Current Ext. Price',
                                               'Valid Proposed Ext. Price'], index = 'Manufacturer', aggfunc = np.sum)

Manufacturer['Percent Increase'] = Manufacturer['$ Difference'] / Manufacturer['Valid Current Ext. Price']
Manufacturer['Percent Increase']= Manufacturer['Percent Increase']*100
Manufacturer= Manufacturer[Manufacturer['$ Difference'].notnull()]

Manufacturer=Manufacturer.rename(columns={'Extended Price':'Spend'})
Manufacturer.sort("Spend", ascending=False)[:6]

,$ Difference,Spend,Quantity,Valid Current Ext. Price,Valid Proposed Ext. Price,Percent Increase
Manufacturer,,,,,,


In [39]:
import datetime as dt
from datetime import datetime
from datetime import date

date = dt.date.today().strftime("%m/%d/%Y")

Summary= df([dict(Supplier=Supplier,Spend=spenddollars,Validity_Percent=valid_percentage,Price_Difference=price_difference_dollars,Percentage_of_Increase=percentage,Quantity_Removed=removed,Quantity_Removed2=removed2,Removed_Spend = Removed_Spend_Dollars, Campus = Campus, Date = date),])

Log = pd.read_csv(LogFile)
Log = Log.append(Summary)
Log.to_csv(LogFile, index = False)